## Part 2  -  VacationPy - Choose Vacation Spots with Ideal Weather

### Introduction 

In VacationPy, I have used the hvplot.pandas plotting library to plot all the cities on a world map. After that, I began searching for a list of cities experiencing excellent weather that would suit a summer holiday. I copied my new dataframe, 'city_data_vac_df', from 'city_data_time_df' restricting 'Max Temp to between 22 and 30, Wind Speed to less than 4.5 m/s and cloudiness less than 5%. I found 13 cities that met my requirements. I used the Geoapify API to find a hotel less than 10 km from the city centre for each city. I printed a map of the cities and added the hotel's name and country to hover information.

Note that both the World Map with all data points, and the Vacation Spot Map take some time generate and plot.

### Load and Set Up Dependcies

#### Using hvplot. pandas to plot maps
##### I also loaded cartopy, geoviews and pyproj libraries using pip install Jupiter Notebook.

In [1]:
# Dependencies and Setup

# !pip install cartopy
# !pip install geoviews
# !pip install puproj

import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import numpy as np

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/citiestime.csv")
city_data_time_df = pd.read_csv(csv_file)

# Display sample data
city_data_time_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Time
0,0,Adamstown,-25.0660,-130.1015,23.99,82,89,3.44,PN,2024-01-03,09:04:25
1,1,Bayanhongor,46.7167,100.1167,-20.34,82,2,1.32,MN,2024-01-03,09:04:25
2,2,Byron Bay,-28.6500,153.6167,24.28,89,6,4.06,AU,2024-01-03,09:04:25
3,3,Constantia,44.1833,28.6500,6.50,78,100,4.65,RO,2024-01-03,09:02:24
4,4,Vavoua,7.3819,-6.4778,27.12,53,5,1.07,CI,2024-01-03,09:04:26


### Creating a map that displays a point for every city in 'city_data_time_df' DataFrame.

#### The size and colour of each point is dependent on the humidity of each location.

In [3]:
# configure the map plot.

# Ensure hvplot is enabled for geographic plotting
hvplot.extension('bokeh')

# Get data acquisition date.
fetch_date = city_data_time_df['Date'][0]

# Configuring the map plot
map_plot = city_data_time_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True,  # Enable geographic coordinates
    size='Humidity',  # Use 'Humidity' for point sizes
    hover_cols=['City', 'Country', 'Humidity'],  # Data to show on hover
    tiles='OSM',  # Use OpenStreetMap tiles
    frame_width=1000,
    frame_height=600,
    color='Humidity',  # Color points based on 'Humidity'
    cmap='inferno',  # Color map
    alpha=0.9,  # Transparency of the points
    xlim=(-180, +180),  # Set the longitude range
    ylim=(-50, +70),     # Set the latitude range
    xlabel='Longitude',  # Label for the X-axis
    ylabel='Latitude',   # Label for the Y-axis
    title=f'City Locations and Humidity  -  {fetch_date}'  # Title of the plot
)
    # display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Narrow down cities in "city_data_time_df" to find a place with ideal weather

 #### Ideal weather deemed to be 22 <= Max Temp <= 30, Wind Speed < 4.5, and Cloudiness = 0

#### Ideal holiday weather deemed to be 22<=Max Temp <=30, Wind Speed <4.5 m/s , Cloudiness <=5%

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_vac_df = city_data_time_df[
    (city_data_time_df['Max Temp'] >= 22) & 
    (city_data_time_df['Max Temp'] <= 30) &
    (city_data_time_df['Wind Speed'] < 4.5) &
    (city_data_time_df['Cloudiness'] <= 5
    )
]

# Drop any rows with null values
city_data_vac_df = city_data_vac_df.dropna()

# Display sample data

city_data_vac_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Time
4,4,Vavoua,7.3819,-6.4778,27.12,53,5,1.07,CI,2024-01-03,09:04:26
102,102,Usta Muhammad,28.1794,68.0445,24.12,17,0,1.23,PK,2024-01-03,09:04:59
107,107,Tura,25.5198,90.2201,23.18,37,0,2.21,IN,2024-01-03,09:05:01
163,163,Brisas De Zicatela,15.8369,-97.0419,23.68,66,1,1.83,MX,2024-01-03,09:05:22
177,177,Camayenne,9.5350,-13.6878,25.88,74,4,1.51,GN,2024-01-03,09:05:29


### Create a new DataFrame called Hotel_df, and add an extra column called 'Hotel Name'

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_vac_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = np.nan

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
4,Vavoua,CI,7.3819,-6.4778,53,NaN
102,Usta Muhammad,PK,28.1794,68.0445,17,NaN
107,Tura,IN,25.5198,90.2201,37,NaN
163,Brisas De Zicatela,MX,15.8369,-97.0419,66,NaN
177,Camayenne,GN,9.5350,-13.6878,74,NaN
209,South Grafton,AU,-29.7000,152.9500,87,NaN
212,Kalamnuri,IN,19.6667,77.3333,29,NaN
223,Agat Village,GU,13.3886,144.6585,86,NaN
233,Sur,OM,22.5667,59.5289,47,NaN
251,Beaufort West,ZA,-32.3567,22.5830,36,NaN


### Find and download close by hotels in each city using Geoapify API.

In [6]:
def find_hotel(lat, lng):
    base_url = "https://api.geoapify.com/v2/places"
    params = {
        'categories': 'accommodation.hotel',
        'filter': f'circle:{lng},{lat},10000',
        'limit': 1,
        'apiKey': geoapify_key
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()

        data = response.json()
        if data['features']:
            hotel_info = data['features'][0]
            return hotel_info['properties']['name']
        else:
            return "No hotel found"
    except: 
        return f"API request error: {e}"

# Looping through the DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    hotel_name = find_hotel(lat, lng)
    hotel_df.at[index, 'Hotel Name'] = hotel_name

hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
4,Vavoua,CI,7.3819,-6.4778,53,SIA Hotel
102,Usta Muhammad,PK,28.1794,68.0445,17,No hotel found
107,Tura,IN,25.5198,90.2201,37,No hotel found
163,Brisas De Zicatela,MX,15.8369,-97.0419,66,Hotel Paraiso Escondido
177,Camayenne,GN,9.5350,-13.6878,74,Palm Camayenne
209,South Grafton,AU,-29.7000,152.9500,87,Great Northern Hotel
212,Kalamnuri,IN,19.6667,77.3333,29,No hotel found
223,Agat Village,GU,13.3886,144.6585,86,Leo Palace
233,Sur,OM,22.5667,59.5289,47,Resort Sur Beach Hotel
251,Beaufort West,ZA,-32.3567,22.5830,36,Wagon Wheel


### For each city, use the Geoapify API to find the first hotel located within 10,000 metres of these coordinates.

In [7]:
def find_hotel(lat, lng):
    base_url = "https://api.geoapify.com/v2/places"
    params = {
        'categories': 'accommodation.hotel',
        'filter': f'circle:{lng},{lat},10000',
        'limit': 1,  # we only want the first result
        'apiKey': geoapify_key
    }
   
    #make request and receive response.
    response = requests.get(base_url, params=params)
        
    if response.status_code == 200:
        data = response.json()
                
        if data['features']:
            # Extract hotel information
            hotel_info = data['features'][0]
            hotel_name = hotel_info['properties']['name']
            return hotel_name
        else:
            return "No hotel found"
    else:
        return "API request failed"
    
# Looping through the DataFrame
for index, row in hotel_df.iterrows():
    # Extracting latitude and longitude for each city
    lat = row['Lat']
    lng = row['Lng']
    
    # Calling the find_hotel function
    hotel_name = find_hotel(lat, lng)
    # Storing the result back in the DataFrame
    hotel_df.at[index, 'Hotel Name'] = hotel_name
    
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
4,Vavoua,CI,7.3819,-6.4778,53,SIA Hotel
102,Usta Muhammad,PK,28.1794,68.0445,17,No hotel found
107,Tura,IN,25.5198,90.2201,37,No hotel found
163,Brisas De Zicatela,MX,15.8369,-97.0419,66,Hotel Paraiso Escondido
177,Camayenne,GN,9.5350,-13.6878,74,Palm Camayenne
209,South Grafton,AU,-29.7000,152.9500,87,Great Northern Hotel
212,Kalamnuri,IN,19.6667,77.3333,29,No hotel found
223,Agat Village,GU,13.3886,144.6585,86,Leo Palace
233,Sur,OM,22.5667,59.5289,47,Resort Sur Beach Hotel
251,Beaufort West,ZA,-32.3567,22.5830,36,Wagon Wheel


### Add hotel name and country to the hover message for each city.

### Add hotel name and country to the hover message for each city, Plot the Map

In [8]:
#%%capture --no-display

# Configure the map plot
# Ensure hvplot is enabled for geographic plotting
hvplot.extension('bokeh')

# Configuring the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True,  # Enable geographic coordinates
    size='Humidity',  # Use 'Humidity' for point sizes
    hover_cols=['City','Country', 'Hotel Name'],  # Data to show on hover
    tiles='OSM',  # Use OpenStreetMap tiles
    frame_width=1000,
    frame_height=500,
    color='Humidity',  # Color points based on 'Humidity'
    cmap='Reds',  # Color map
    alpha=0.9,  # Transparency of the points
    xlim=(-110, 60),
    ylim=(-40, 25),
    xlabel='Longitude',  # Label for the X-axis
    ylabel='Latitude',   # Label for the Y-axis
    title=f'Potential Vacation Hotel Locations  -  {fetch_date}'  # Title of the plot
)
    
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)

### My Perfect Holiday Places

#### Our program found 21 differenct cities with the desired weather.  It looks like the perfect holiday destination in January 2024 for a beach loving Australian looking for almost cloudless skys could be in serveral different countries.  Note that, the maximum temperature we use in the calculation, can depend on the time day in which re retreive the weather data from OpenWeatherMaps.Org.